## Quantidade de cluster

In [0]:
%sql
WITH distinct_count AS (
  SELECT COUNT(DISTINCT cluster_id) AS distinct_cluster_id_count
  FROM system.compute.clusters
  WHERE workspace_id = '1942930051229702'
)
SELECT 
  cluster_id,
  dc.distinct_cluster_id_count
FROM system.compute.clusters sc
CROSS JOIN distinct_count dc
WHERE sc.workspace_id = '1942930051229702'
  AND (
    (sc.cluster_name LIKE 'job%' AND sc.create_time = (
      SELECT MAX(create_time)
      FROM system.compute.clusters
      WHERE workspace_id = '1942930051229702'
    ))
    OR sc.cluster_name NOT LIKE 'job%'
  )

## Consumo de DBU

In [0]:
%sql
SELECT 
    ROUND(CAST(SUM(u.usage_quantity) AS FLOAT), 2) AS total_usage_quantity
FROM `system`.billing.usage u
WHERE u.workspace_id = '1942930051229702';

## Utilização de CPU

In [0]:
%sql
SELECT 
  cluster_id,
  DATE(start_time) AS start_time,
  MAX(cpu_user_percent) AS max_cpu_user_percent
FROM system.compute.node_timeline
WHERE workspace_id = '1942930051229702'
  AND cluster_id IN (
    SELECT cluster_id
    FROM system.compute.clusters
    WHERE workspace_id = '1942930051229702'
      AND (
        (cluster_name LIKE 'job%' AND create_time = (
          SELECT MAX(create_time)
          FROM system.compute.clusters
          WHERE workspace_id = '1942930051229702'
        ))
        OR cluster_name NOT LIKE 'job%'
      )
  )
GROUP BY cluster_id, DATE(start_time)
ORDER BY cluster_id, start_time

## Custo de utilização de cluster

In [0]:
%sql
SELECT 
    CAST(SUM(u.usage_quantity * lp.default) AS FLOAT) AS total_cost_cluster
FROM `system`.billing.usage u
LEFT JOIN (
    SELECT 
        sku_name,
        pricing.default,
        DATE(price_start_time) AS price_start_date
    FROM system.billing.list_prices
    WHERE (sku_name, price_start_time) IN (
        SELECT 
            sku_name, 
            MAX(price_start_time) AS max_price_start_time
        FROM system.billing.list_prices
        GROUP BY sku_name
    )
) lp
ON u.sku_name = lp.sku_name
WHERE u.workspace_id = '1942930051229702'

## Custo por SKU

In [0]:
%sql
SELECT 
    u.sku_name,
    CAST(SUM(u.usage_quantity * lp.default) AS FLOAT) AS total_cost_cluster
FROM `system`.billing.usage u
LEFT JOIN (
    SELECT 
        sku_name,
        pricing.default,
        DATE(price_start_time) AS price_start_date
    FROM system.billing.list_prices
    WHERE (sku_name, price_start_time) IN (
        SELECT 
            sku_name, 
            MAX(price_start_time) AS max_price_start_time
        FROM system.billing.list_prices
        GROUP BY sku_name
    )
) lp
ON u.sku_name = lp.sku_name
WHERE u.workspace_id = '1942930051229702'
GROUP BY u.sku_name
ORDER BY total_cost_cluster DESC


## Evolução de utilização de DBU

In [0]:
%sql
SELECT 
    DATE(u.usage_end_time) AS usage_end_date,
    ROUND(CAST(SUM(u.usage_quantity) AS FLOAT), 2) AS total_dbu_usage
FROM `system`.billing.usage u
WHERE u.workspace_id = '1942930051229702'
  AND DATE(u.usage_end_time) >= DATE_ADD(CURRENT_DATE, -7)
GROUP BY DATE(u.usage_end_time)
ORDER BY usage_end_date ASC

## Lista de cluster

In [0]:
%sql
SELECT 
cluster_name, 
create_time,
change_time, 
driver_node_type, 
worker_node_type, 
dbr_version,  
data_security_mode
FROM `system`.compute.clusters
WHERE workspace_id = '1942930051229702'
  AND (cluster_name LIKE 'job%' AND create_time = (SELECT MAX(create_time) FROM `system`.compute.clusters WHERE workspace_id = '1942930051229702') OR cluster_name NOT LIKE 'job%')